In [79]:
import pandas as pd

# Load the uploaded data files
inj_data = pd.read_csv('inj-usd-max.csv')
bico_data = pd.read_csv('bico-usd-max.csv')
auction_data = pd.read_csv('injective_auction_data.csv')
market_cap_data = pd.read_csv('CG_market_cap.csv')

# Display the first few rows of each dataframe to understand their structure
inj_data_head = inj_data.head()
bico_data_head = bico_data.head()
auction_data_head = auction_data.head()
market_cap_data_head = market_cap_data.head()


In [80]:
# Cleaning and processing the data

# 1. Convert date columns to datetime format and clean auction data
inj_data['Unnamed: 0'] = pd.to_datetime(inj_data['Unnamed: 0'], format='%Y-%m-%d %H:%M:%S UTC')
bico_data['Unnamed: 0'] = pd.to_datetime(bico_data['Unnamed: 0'], format='%Y-%m-%d %H:%M:%S UTC')
auction_data['UTC Timestamp'] = pd.to_datetime(auction_data['UTC Timestamp'], format='%m/%d/%y %H:%M')

# Clean up 'INJ Amount' and 'USD Amount' by removing text and converting to floats
auction_data['INJ Amount'] = auction_data['INJ Amount'].str.replace(' INJ', '')
auction_data['INJ Amount'] = auction_data['INJ Amount'].str.replace(',', '').astype(float)
auction_data['USD Amount'] = auction_data['USD Amount'].str.replace(r'[\$, usd]', '', regex=True).astype(float)

# 2. Rename columns for better clarity
inj_data.columns = ['Date', 'Price', 'Market Cap']
bico_data.columns = ['Date', 'Price', 'Market Cap']
auction_data.columns = ['Date', 'INJ Amount', 'USD Amount']
market_cap_data['UTC Time'] = pd.to_datetime(market_cap_data['UTC Time'], format='%m/%d/%y %H:%M')
market_cap_data.columns = ['Date', 'Altcoin Market Cap']

In [84]:
# Cumulative return calculation formula: (Price_t / Price_0) - 1

# 1. Injective Cumulative Return
inj_data['Injective Cumulative Return'] = (inj_data['Price'] / inj_data['Price'].iloc[0]) - 1

# 2. Biconomy Cumulative Return (from bico_data)
bico_data['Biconomy Cumulative Return'] = (bico_data['Price'] / bico_data['Price'].iloc[0]) - 1

# 3. Altcoin Market Cap Cumulative Change (used as a benchmark)
market_cap_data['Altcoin Cumulative Change'] = (market_cap_data['Altcoin Market Cap'] / market_cap_data['Altcoin Market Cap'].iloc[0]) - 1

inj_data['Date'] = pd.to_datetime(inj_data['Date']).dt.floor('D')
bico_data['Date'] = pd.to_datetime(bico_data['Date']).dt.floor('D')
market_cap_data['Date'] = pd.to_datetime(market_cap_data['Date']).dt.floor('D')

In [89]:

long_term_comparison = pd.merge(inj_data[['Date', 'Injective Cumulative Return']], 
                                bico_data[['Date', 'Biconomy Cumulative Return']], 
                                on='Date', how='inner')

long_term_comparison = pd.merge(long_term_comparison, 
                                market_cap_data[['Date', 'Altcoin Cumulative Change']], 
                                on='Date', how='inner')

auction_data_sorted = auction_data.sort_values('Date')
long_term_comparison['Cumulative Burned Tokens'] = 0

for index, row in long_term_comparison.iterrows():
    # For each date in market_cap_comparison, calculate cumulative INJ burned up to that date
    cumulative_burn = auction_data_sorted[auction_data_sorted['Date'] <= row['Date']]['INJ Amount'].sum()
    
    # Store the cumulative burn amount in the corresponding row
    long_term_comparison.at[index, 'Cumulative Burned Tokens'] = cumulative_burn

long_term_comparison

Date  Injective Cumulative Return  Biconomy Cumulative Return  \
0   2021-12-01                    16.729975                    0.000000   
1   2021-12-02                    15.613798                    0.000000   
2   2021-12-03                    15.258550                   -0.339254   
3   2021-12-04                    14.515490                   -0.447234   
4   2021-12-05                    12.787822                   -0.484752   
..         ...                          ...                         ...   
625 2024-09-14                    24.861632                   -0.984024   
626 2024-09-16                    23.365136                   -0.984770   
627 2024-09-18                    24.363206                   -0.984790   
628 2024-09-20                    25.758062                   -0.983532   
629 2024-09-22                    26.599645                   -0.982972   

     Altcoin Cumulative Change  Cumulative Burned Tokens  
0                    10.411495                      0.00  
1                    10.211975                      0.00  
2                    10.210302                      0.00  
3                     9.454792                      0.00  
4                     8.938022                      0.00  
..                         ...                       ...  
625                   6.096895                6209790.88  
626                   5.697321                6209790.88  
627                   5.819335                6209790.88  
628                   5.963302                6220277.10  
629                   6.241532                6220277.10  

[630 rows x 5 columns]

In [90]:
# Importing statsmodels.api (sm) again to resolve the missing import
import statsmodels.api as sm




# Re-running the regression setup
X = long_term_comparison[['Cumulative Burned Tokens', 'Altcoin Cumulative Change']]
y = long_term_comparison['Injective Cumulative Return']

# Adding a constant to the independent variables for the regression intercept
X = sm.add_constant(X)

# Run the regression model
model = sm.OLS(y, X).fit()

# Display the summary of the regression results
model_summary = model.summary()
model_summary


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     Injective Cumulative Return   R-squared:                       0.636
Model:                                     OLS   Adj. R-squared:                  0.635
Method:                          Least Squares   F-statistic:                     548.5
Date:                         Wed, 25 Sep 2024   Prob (F-statistic):          1.93e-138
Time:                                 15:37:05   Log-Likelihood:                -2254.3
No. Observations:                          630   AIC:                             4515.
Df Residuals:                              627   BIC:                             4528.
Df Model:                                    2                                         
Covariance Type:                     nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                       -19.4427      1.103    -17.619      0.000     -21.610     -17.276
Cumulative Burned Tokens   3.636e-06   1.31e-07     27.843      0.000    3.38e-06    3.89e-06
Altcoin Cumulative Change     4.0445      0.173     23.417      0.000       3.705       4.384
==============================================================================
Omnibus:                      145.604   Durbin-Watson:                   0.037
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              277.585
Skew:                           1.323   Prob(JB):                     5.29e-61
Kurtosis:                       4.890   Cond. No.                     1.36e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.36e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""